### Recently I published a self help book titled Inspiration: Thoughts on Spirituality, Technology, Wealth, Leadership and Motivation. The preview of the book can be read from the Amazon link https://lnkd.in/gj7bMQA

### You can refer to my other notebooks from https://www.kaggle.com/binuthomasphilip/code

In an machine learning problem one of the most import steps is feature selection.Feature selection helps in simplification of model,improves accuracy,reduces training time,reduces overfitting and helps in avoiding curse of dimentionality.Here we will cover different techniques like

1.Univariate feature Selection using p value and f Distribution

2.Types of Univarate Feature Selection 

a)Select K Best

b)Select Percentile

c)Generic Univariate Select

3.Tree Based using Random Forest

4.Regularization using Lasso

5.Conclusion 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Importing Data Set 

In [ ]:
df = pd.read_csv("../input/breast-cancer-wisconsin-data/data.csv")
df.head(2)

### Dropping Columns

In [ ]:
df = df.drop(['id','diagnosis','Unnamed: 32'],axis=1) # Dropping Multiple Columns
df.head(2)

We have dropped the columns which will not be useful for us during demonstration of feature selection techniques.

### Splitting data into Independent and Dependent Features

In [ ]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

Here we are taking all the feature except fractal_dimension_worst as independent variables and fractal_dimension_worst is considered as dependent variables.

### Test Train Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state= 10)

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,y_train)
y_predict = lr.predict(X_test)

### Mean Square Error

In [ ]:
from sklearn.metrics import mean_squared_error
import math
rsme = math.sqrt(mean_squared_error(y_test,y_predict))
rsme

# 1.Univariate Feature Selection using f Distribution

In [ ]:
from sklearn.feature_selection import f_regression as fr
result = fr(X,y)
f_score = result[0]
p_values = result[1]

# Getting column names 
columns = list(X.columns)
print(" ")
print(" ")
print(" ")

print("     Features                     ","F-Score  ","P-Values")
print("     ------------                   --------  ---------")

for i in range(0,len(columns)):
    f1 = "%4.2f" % f_score[i]
    p1 = "%2.6f" % p_values[i]
    print("    ",columns[i].ljust(25),f1.rjust(12),"",p1.rjust(8))

So now we have the F_score and p_values for all the features.For a feature to be statistically significant the p values should be less than 5% ie 0.05.If for any feature the p values is more than 0.05 we can discard the feature as it wont be statistically significant for predicting the dependent feature.

## Model Performance With Selected Features

### Updated Matrix of features

In [ ]:
X_train_n = X_train[['texture_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','perimeter_se','smoothness_se','compactness_se','concavity_se','concave points_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst']]
X_test_n = X_test[['texture_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','perimeter_se','smoothness_se','compactness_se','concavity_se','concave points_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst']]

### Updated Linear Model

In [ ]:
from sklearn.linear_model import LinearRegression
lr1 = LinearRegression()
lr1.fit(X_train_n,y_train)
y_predict1 = lr1.predict(X_test_n)

In [ ]:
from sklearn.metrics import mean_squared_error
import math
rsme1 = math.sqrt(mean_squared_error(y_test,y_predict1))
rsme1

Doing a feature selection based on p value of 0.05 has increased rsme value.So it tells us that some features having p value more than 0.05 have statistical significance on the outcome of the prediction.So we can either change the p value or include all the features while predicting the outcome.

### You can refer to my other notebooks from https://www.kaggle.com/binuthomasphilip/code

# 2.Types of Univariate Feature Selection

### a)Select K Best

In this case we will be specifying the value of K which represents the number of best features we want to select.

In [ ]:
from sklearn.feature_selection import f_regression as fr
from sklearn.feature_selection import SelectKBest
selectorK = SelectKBest(score_func=fr,k=5)
X_k = selectorK.fit_transform(X,y)
#X_k

In [ ]:
# Getting f_score and p values for the selected features
#f_score = X_k[0]
#p_values = X_k[1]

f_score = selectorK.scores_
p_values = selectorK.pvalues_

# Getting column names 
columns = list(X.columns)
print(" ")
print(" ")
print(" ")

print("     Features                     ","F-Score  ","P-Values")
print("     ------------                   --------  ---------")

for i in range(0,len(columns)):
    f1 = "%4.2f" % f_score[i]
    p1 = "%2.6f" % p_values[i]
    print("    ",columns[i].ljust(25),f1.rjust(12),"",p1.rjust(8))

In [ ]:
# get the column names 
cols = selectorK.get_support(indices = True)
selectedCols = X.columns[cols].tolist()
print(selectedCols)

So if we were to selected 5 best features for our prediction then they will be the five features desplayed above.They are based on five lowest p values which is less than 0.05

### b)Select Percentile

In [ ]:
from sklearn.feature_selection import f_regression as fr
from sklearn.feature_selection import SelectPercentile
selectorP = SelectPercentile(score_func=fr,percentile=20)
X_p = selectorP.fit_transform(X,y)

In [ ]:
# get the column names 
cols_p = selectorP.get_support(indices = True)
selectedCols_p = X.columns[cols_p].tolist()
print(selectedCols_p)

So we have selected 20 % of the important features in our dataset.So we get a list of 6 most important features.

### c)Generic Univariate Select

#### Based on K Best

In [ ]:
from sklearn.feature_selection import f_regression as fr
from sklearn.feature_selection import GenericUnivariateSelect
selectorG1 = GenericUnivariateSelect(score_func=fr,mode='k_best',param=3)
X_g1 = selectorG1.fit_transform(X,y)

In [ ]:
# get the column names 
cols_g1 = selectorG1.get_support(indices = True)
selectedCols_g1 = X.columns[cols_g1].tolist()
print(selectedCols_g1)

#### Based on Percentile

In [ ]:
from sklearn.feature_selection import f_regression as fr
from sklearn.feature_selection import GenericUnivariateSelect
selectorG2 = GenericUnivariateSelect(score_func=fr,mode='percentile',param=20)
X_g2 = selectorG2.fit_transform(X,y)

In [ ]:
# get the column names 
cols_g2 = selectorG2.get_support(indices = True)
selectedCols_g2 = X.columns[cols_g2].tolist()
print(selectedCols_g2)

So we have covered different Univariate Feature selection techiniques in the above sections.

# 3.Tree Based

In [ ]:
from sklearn.ensemble import RandomForestRegressor
np.random.seed()
forest = RandomForestRegressor(n_estimators=1000)
fit = forest.fit(X_train,y_train)
accuracy = fit.score(X_test,y_test)
predict = fit.predict(X_test)
#cmatrix = confusion_matrix(y_test,predict)

#-------------------------------------------------------------------------------------------------#
# Perform k Fold cross- validation 

print('Accuracy of Random Forest: %s'% "{0:.2%}".format(accuracy))

### Feature Importance 

In [ ]:
# Feature importance 
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
print("Feature ranking")
for f in range(X.shape[1]):
    print("Feature %s (%f)" % (list(X)[f],importances[indices[f]]))

feat_imp = pd.DataFrame({'Feature':list(X),
                        'Gini importance':importances[indices]})
plt.rcParams['figure.figsize']=(12,12)
sns.set_style('whitegrid')
ax = sns.barplot(x='Gini importance',y='Feature',data=feat_imp)
ax.set(xlabel='Gini Importance')
pass

Using the Random Forest Regressor we have done feature selection based on their importance.

# 4.Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso 
from sklearn.feature_selection import SelectFromModel
sel_ = SelectFromModel(Lasso(alpha=0.005,random_state=0))
sel_.fit(X_train,y_train)

In [ ]:
sel_.get_support()

In [ ]:
selected_feat=X_train.columns[(sel_.get_support())]

print('total features: {}'.format((X_train.shape[1])))
print('Selected features: {}'.format(len(selected_feat)))
print('Features with coefficients shrank to zero: {}'.format(np.sum(sel_.estimator_.coef_==0)))

In [ ]:
selected_feat

In [ ]:
selected_feat = X_train.columns[(sel_.estimator_.coef_!=0).ravel().tolist()]
selected_feat

We have see how we are able to do a feature slection using Lasso Regression.By changing the value of alpha we will be ablw to get desired number of featured for our ML Model.

# 5.Conclusion 

In the Kernel we have covered Univariate,Tree based and Lasso feature selection techniques for Regression Problems,I hope this kernel will be useful for you in future to carry out activity of feature selection on your dataset.

### You can refer to my other notebooks from https://www.kaggle.com/binuthomasphilip/code

# TO BE CONTINUED